In [32]:
import os
import json
import pandas as pd
import numpy as np
from datetime import datetime
from casstools.cass_client import CassClient

In [2]:
client = CassClient()

In [3]:
def initialize_directories(basepath="."):
    directories = ['blackboard', 'fetched', 'templates']
    for d in directories:
        pathspec = os.path.join(basepath,d)
        if not os.path.exists(pathspec):
            os.makedirs(pathspec)

In [68]:
def remove_time_offset(time_string_iso_with_offset: str, timezone: str) -> str:
    from zoneinfo import ZoneInfo
    dt = datetime.fromisoformat(time_string_iso_with_offset).astimezone(ZoneInfo("America/New_York"))
    dts = dt.strftime("%Y-%m-%d %H:%M")
    return dts

In [78]:
def fetch_assignments(course_key, assignment_name) -> pd.DataFrame:
    submissions = []
    me = client.whoami()
    assignments = client.get_course_assignments(course_key)
    roster = client.get_course_roster(course_key)
    assignment_dict = assignments[ assignments['assignment_name'] == assignment_name].to_dict(orient="records")[0]
    student_list = roster['student'].tolist()
    folder = f"./fetched/{course_key}/{me['name']}/{assignment_dict['unit_name']}/{assignment_dict['assignment_name']}"
    os.makedirs(folder, exist_ok=True)

    for student_name in student_list:
        row = { 'student' : student_name }
        info = client.submission_info(course_key, assignment_name, student_name, assignment_name)
        infofile = f"{folder}/{student_name}-info.json"
        with open(infofile,"w") as f:
            f.write(json.dumps(info))
        if info['exists']:
            row['submitted_on'] = remove_time_offset(info['last_modified'], "America/New_York")
            row['etag'] = info['etag']
            #fetch the file!
            notebook = client.get_assignment_submission(course_key, assignment_name, student_name, assignment_name)
            notebookfile = f"{folder}/{student_name}.ipynb"
            row['notebookfile'] = notebookfile
            with open(notebookfile, "w") as f:
                f.write(json.dumps(notebook))
        else:
            row['submitted_on'] = None
            row['etag'] = None
            
        row['late'] = None if row['submitted_on'] is None else datetime.fromisoformat(row['submitted_on']) > datetime.fromisoformat(assignment_dict['due_date'])
        row['due_date'] = assignment_dict['due_date']
        row['total_points'] = assignment_dict['points']
        row['infofile'] = infofile
        submissions.append(row)
    
    print(assignment_dict)
    return submissions


In [79]:
submissions = fetch_assignments("tst101-spring2024", "A2.ipynb")
df = pd.DataFrame(submissions)

{'unit_name': 'A', 'assignment_name': 'A2.ipynb', 'points': 5, 'due_date': '2024-01-06 23:59', 'lms_column_id': 342123}


In [80]:
def make_clickable(url, name):
    if url is np.nan:
        return name
    else:
        return '<a href="{}" rel="noopener noreferrer">{}</a>'.format(url,name)

df['link'] = df.apply(lambda row: make_clickable(row['notebookfile'], row['student'] ), axis=1)
df[['link','student','submitted_on','late', 'due_date']].style


,link,student,submitted_on,late,due_date
0,mafudge@syr.edu,mafudge@syr.edu,2024-01-06 20:03,False,2024-01-06 23:59
1,dlnosky@syr.edu,dlnosky@syr.edu,None,None,2024-01-06 23:59


In [58]:
def remove_time_offset(time_string_iso_with_offset: str, timezone: str) -> datetime:
    from zoneinfo import ZoneInfo
    dt = datetime.fromisoformat(time_string_iso_with_offset).astimezone(ZoneInfo("America/New_York"))
    dts = dt.strftime("%Y-%m-%d %H:%M")
    return dts


remove_time_offset(mds, "America/New_York")


datetime.datetime(2024, 1, 6, 20, 3)

In [59]:
minio_submission = "2024-01-07T01:03:16.895000+00:00"
duedate = "2024-01-06 23:59"
minio_submission_dt = remove_time_offset(minio_submission, "America/New_York")
duedate_dt =  datetime.fromisoformat(d2)

In [61]:
minio_submission_dt, duedate_dt, minio_submission_dt < duedate_dt

(datetime.datetime(2024, 1, 6, 20, 3),
 datetime.datetime(2024, 1, 6, 23, 59),
 True)

datetime.datetime(2024, 1, 6, 20, 3, 16, 895000, tzinfo=zoneinfo.ZoneInfo(key='America/New_York'))